# These function are what we used to transform the data in making the Model.
### We are going to use these function to transform any data we want to predict in the microservice

In [ ]:
import pandas as pd
import re
import math 
import datetime

###### Dependencies

In [ ]:
arabic_numbers = '٠١٢٣٤٥٦٧٨٩'
english_numbers = '0123456789'
translate_arabic_numbers = str.maketrans(arabic_numbers, english_numbers)

def get_number_from_string(string):
   numbers = re.findall(r'\d+', string)
   total = sum(int(num) for num in numbers)
   return total

kilometers_per_liter = {
"بنزين":{
        "less":10.8,
        "more":7.7
},"ديزل":{
        "less":9.7,
        "more":5.6
}
}

amount_of_fuel_per_month = {
  "بنزين":{
        "less":159,
        "more":270
},"ديزل":{
        "less":389,
        "more":841
}  
}
regex_mapping = {
    "شرك":1,
    "صفر":0,
    "اول":1,
    "أول":1,
    "ثان":2,
    "تان":2,
    "ثال":3,
    "تال":3,
    "راب":4,
    "خام":5,
    "ساد":6,
    "ساب":7,
    "ثام":8,
    "تام":8,
    "تاس":9,
    "عاش":10,
    "مست":1,
    "غير":0,
    "مش":0,
    "نفس":1,
}

mean_kilometers_driven = 192151.4508096547
std_kilometers_driven = 186232.66769881285

###### Dependant

In [1]:
additions_features = ['مسجل','مُكيّف','مركزي','إغلاق','مغنيسيوم','هوائية','وسادة','جنطات','جهاز','إنذار','جلد','فرش','سقف','فتحة']

compound_words_additions_features = {
    'إغلاق مركزي':['إغلاق','مركزي'],
    'جنطات مغنيسيوم':['جنطات','مغنيسيوم'],
    'وسادة هوائية':['وسادة','هوائية'],
    'جهاز إنذار':['جهاز','إنذار'],
    'فرش جلد':['فرش','جلد'],
    'فتحة سقف':['فتحة','سقف']
}

def extract_num_seats(value):
    if get_number_from_string(str(value).translate(translate_arabic_numbers)):
        return get_number_from_string(str(value).translate(translate_arabic_numbers))
    return 5

def convert_car_meter(value):

    number = re.findall(r'\d+', str(value))

    if(not number):
        return None
    number = number[0]
    
    if (float(number) / 1000) >=1:
        return int(number)
    
    return int(number)*1000

today = datetime.date.today()
year = today.year
fuel_keys = ["بنزين","ديزل"]

def estimate_car_meter(df):
   for index, row in df.iterrows():
      if math.isnan(row["عداد السيارة"]) or row["عداد السيارة"] == 0:
         new_value = int()
         cc = int()
         if row["قوة الماتور"] > 2500:
               cc = "more"
         else:
               cc = "less"
         if not row["نوع الوقود"].strip() in fuel_keys:
               new_value = int((year - row["سنة"])*12*1500)
         else:
               Kvalue = kilometers_per_liter[row["نوع الوقود"].strip()][cc]
               Fvalue = amount_of_fuel_per_month[row["نوع الوقود"].strip()][cc]
               denominator = int(row["سنة"]/995)
               new_value = int(((year - row["سنة"])*12*Fvalue*Kvalue)/denominator)
         df.at[index, "عداد السيارة"] = new_value

def convert_previous_owners_to_num(df):
    for index, row in df.iterrows():
        new_value = None
        text = row["أصحاب سابقون"]
        string = str(text).translate(translate_arabic_numbers)
        for key,value in regex_mapping.items():
            match = re.search(key,string)
            if(match):
                new_value = value
                continue
        if get_number_from_string(string) > 0:   
            new_value = get_number_from_string(string)
        else:
             new_value = int((year - row["سنة"])/10)
        df.at[index, "أصحاب سابقون"] = new_value

columns_names_arabic_english = {
    "سعر":"Price",
    "شركة":"Brand",
    "موديل":"Model",
    "سنة":"Year",
    "لون السيارة":"Color",
    "نوع الوقود":"Fuel_Type",
    "أصل السيارة":"Vehicle_History",
    "رخصة السيارة":"Licence",
    "نوع الجير":"Transmission_Type",
    "الزجاج":"Windows_Type",
    "قوة الماتور":"Engine_Capacity",
    "عداد السيارة":"Kilometers_Driven",
    "عدد الركاب":"Seats",
    "وسيلة الدفع":"Payment_Method",
    "معروضة":"Listed_For",
    "أصحاب سابقون":"Previous_Owners",
    "مسجل":"Radio",
    "CD":"CD",
    "مُكيّف":"Air_Conditioner",
    "إغلاق مركزي":"Central_Lock",
    "وسادة هوائية":"Airbag",
    "جنطات مغنيسيوم":"Magnesium_Rims",
    "جهاز إنذار":"Alert_System",
    "فرش جلد":"Leather_Seats",
    "فتحة سقف":"Sunroof"
}

licence_map = {
    "فلسطينية": 1,
    "نمرة صفراء": 0
}

windows_type_map = {
    "الكتروني": 1,
    "يدوي": 0
}

payment_method_map = {
    "نقدا فقط": 0,
    "إمكانية التقسيط": 1
}

columns_rename_map = {
    "Licence":"Palestinian_Licence",
    "Windows_Type":"Electric_Windows",
    "Payment_Method":"Installment_Payment_Method"
}

def convert_kilometers_driven_to_zscore(value):
    return ((value - mean_kilometers_driven)/std_kilometers_driven)

def bin_previous_owners(df):
   return pd.cut(df, bins=[-1, 2, 4, 100], labels=[1, 2, 3])